### 分類任務
使用bert-base-chinese模型對新聞資料集做embeddings，接著訓練分類器。（參考week7程式碼）

In [ ]:
import os

from google.colab import drive
drive.mount('/content/drive')

os.chdir('你的雲端資料夾路徑') #切換該目錄
os.listdir() #確認目錄內容

In [ ]:
import pandas as pd
import re

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [ ]:
!pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer, models, util

In [ ]:
# 中文 bert-base-chinese
bert_ch = SentenceTransformer('google-bert/bert-base-chinese')

bert_ch.tokenizer.add_special_tokens({'pad_token': '[PAD]'})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

In [ ]:
udn = pd.read_csv("./raw_data/udn_news_202402.csv")
udn.head(3)

,system_id,artTitle,artDate,artCatagory,artUrl,artContent,dataSource
0,1,中職／非樹倒猢猻散而是桃李滿天下 戰力外溢彰顯樂天深度,2024-02-01 07:00:00,運動,https://udn.com/news/story/122629/7746474,由於過去幾年在季後的休賽期間總是在人員調動有令人吃驚的動作，因此富邦被戲稱為季後賽王者，沒想...,UDN
1,2,中職／2023驚奇一年 味全龍「紅色大砲」劉基鴻轟出神劇本,2024-02-05 12:00:00,運動,https://udn.com/news/story/122629/7748599,【撰文・鍾亞芳】2021年就曾因骨裂影響出賽，2022季末遭觸身球擊中手腕造成骨裂傷勢，賽季...,UDN
2,3,中職／花店老闆上工 前中信兄弟「轉運手」官大元展開第2人生,2024-02-15 12:00:00,運動,https://udn.com/news/story/122629/7748779,【撰文・鍾亞芳】生涯中大半時間都在思索褪下球員身分後出路，直到2019年收回出租房屋，整理、...,UDN


In [ ]:
# 過濾 nan 的資料
udn = udn.dropna(subset=['artTitle'])
udn = udn.dropna(subset=['artContent'])
# 移除網址格式
udn["artContent"] = udn.artContent.apply(
    lambda x: re.sub("(http|https)://.*", "", x)
)
udn["artTitle"] = udn["artTitle"].apply(
    lambda x: re.sub("(http|https)://.*", "", x)
)
# 只留下中文字
udn["artContent"] = udn.artContent.apply(
    lambda x: re.sub("[^\u4e00-\u9fa5]+", "", x)
)
udn["artTitle"] = udn["artTitle"].apply(
    lambda x: re.sub("[^\u4e00-\u9fa5]+", "", x)
)

# 留下 content
udn["content"] = udn["artTitle"] + udn["artContent"]
udn = udn[["content", "artUrl", "artCatagory"]]  # 文章內容 文章連結
udn.head()

,content,artUrl,artCatagory
0,中職非樹倒猢猻散而是桃李滿天下戰力外溢彰顯樂天深度由於過去幾年在季後的休賽期間總是在人員調動...,https://udn.com/news/story/122629/7746474,運動
1,中職驚奇一年味全龍紅色大砲劉基鴻轟出神劇本撰文鍾亞芳年就曾因骨裂影響出賽季末遭觸身球擊中手腕...,https://udn.com/news/story/122629/7748599,運動
2,中職花店老闆上工前中信兄弟轉運手官大元展開第人生撰文鍾亞芳生涯中大半時間都在思索褪下球員身分...,https://udn.com/news/story/122629/7748779,運動
3,中職冬盟展現打擊天賦統一獅何恆佑潛力股衝刺一軍撰文鄧心瑜何恆佑是教練團公認訓練認真態度積極選...,https://udn.com/news/story/122629/7748904,運動
4,尼克無懼逆境加速向前衝攻防都進入精英級別紐約尼克最近八連勝勝敗站上東區第三尼克進攻效率第六防...,https://udn.com/news/story/122629/7749189,運動


In [ ]:
udn["embeddings"] = udn.content.apply(lambda x: bert_ch.encode(x))
udn.head(3)

,content,artUrl,artCatagory,embeddings
0,中職非樹倒猢猻散而是桃李滿天下戰力外溢彰顯樂天深度由於過去幾年在季後的休賽期間總是在人員調動...,https://udn.com/news/story/122629/7746474,運動,"[0.25986034, -0.10333683, -0.13030435, 0.32271..."
1,中職驚奇一年味全龍紅色大砲劉基鴻轟出神劇本撰文鍾亞芳年就曾因骨裂影響出賽季末遭觸身球擊中手腕...,https://udn.com/news/story/122629/7748599,運動,"[0.38988355, 0.013849409, -0.10238463, 0.16257..."
2,中職花店老闆上工前中信兄弟轉運手官大元展開第人生撰文鍾亞芳生涯中大半時間都在思索褪下球員身分...,https://udn.com/news/story/122629/7748779,運動,"[0.48618165, -0.15005013, 0.004792653, 0.27161..."


In [ ]:
import numpy as np
from ast import literal_eval

In [ ]:
data = udn.copy()

X = data["embeddings"].apply(pd.Series)
y = data["artCatagory"]

# 把整個資料集七三切
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=777
)

print(X_train.head())
print(y_train.head())

           0         1         2         3         4         5         6    \
2139  0.419848 -0.011115 -0.354665  0.067184 -0.004627 -0.040697 -0.055879   
1916  0.376304  0.039414 -0.362329 -0.045580  0.104265 -0.096845 -0.315971   
258   0.480350  0.020603 -0.117614  0.065926  0.057779 -0.239893  0.021461   
2803  0.124938 -0.156221 -0.432256  0.209314  0.108994 -0.008472 -0.243737   
2089  0.427623 -0.042018 -0.351606  0.131566 -0.150872  0.191713  0.023241   

           7         8         9    ...       758       759       760  \
2139 -0.024846 -0.115148 -0.406072  ...  0.175395 -0.039127  0.211727   
1916  0.175212 -0.213345 -0.079263  ... -0.091282 -0.340814  0.215514   
258  -0.129073 -0.147012 -0.332668  ...  0.039450 -0.261835  0.050737   
2803  0.005351 -0.214306 -0.282695  ...  0.045290 -0.246024  0.114624   
2089 -0.288580 -0.281559 -0.295655  ...  0.172587 -0.029071  0.107264   

           761       762       763       764       765       766       767  
2139  0.026978 

In [ ]:
clf = LogisticRegression()
clf.fit(X_train, y_train)
clf

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
y_pred = clf.predict(X_test)
y_pred_proba = clf.predict_proba(X_test)
print(y_pred[:10])

['兩岸' '運動' '運動' '運動' '兩岸' '兩岸' '全球' '全球' '全球' '全球']


In [ ]:
from sklearn.metrics import classification_report

In [ ]:
## Accuracy, Precision, Recall, F1-score
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          全球       0.93      0.95      0.94       451
          兩岸       0.96      0.94      0.95       518
          股市       0.98      0.99      0.99       616
          運動       0.99      0.98      0.99       375

    accuracy                           0.97      1960
   macro avg       0.97      0.97      0.97      1960
weighted avg       0.97      0.97      0.97      1960

